In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import accuracy_score
from imblearn.combine import SMOTEENN

In [139]:
df = pd.read_csv(r'F:\analytics\churn analysis\ChurnAnalysisProject-PowerBI-main/Telco-Customer-Churn.csv')

In [140]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [141]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
df.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [7]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [142]:
df.TotalCharges= pd.to_numeric(df.TotalCharges,errors = 'coerce')

In [143]:
# Group the tenure in bins of 12 months
labels = ["{0} - {1}".format(i, i + 11) for i in range(1, 72, 12)]

df['tenure_group'] = pd.cut(df.tenure, range(1, 80, 12), right=False,labels=labels)

In [144]:
df['tenure_group'].value_counts()

1 - 12     2175
61 - 72    1407
13 - 24    1024
49 - 60     832
25 - 36     832
37 - 48     762
Name: tenure_group, dtype: int64

In [145]:
df['Churn'] = np.where(df.Churn == 'Yes',1,0)

In [146]:
df.drop('tenure',axis=1,inplace=True)

In [147]:
df.dropna(how='any',axis=0,inplace = True)

In [148]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_group
0,7590-VHVEG,Female,0,Yes,No,No,No phone service,DSL,No,Yes,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,1 - 12
1,5575-GNVDE,Male,0,No,No,Yes,No,DSL,Yes,No,...,No,No,No,One year,No,Mailed check,56.95,1889.50,0,25 - 36
2,3668-QPYBK,Male,0,No,No,Yes,No,DSL,Yes,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1,1 - 12
3,7795-CFOCW,Male,0,No,No,No,No phone service,DSL,Yes,No,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,37 - 48
4,9237-HQITU,Female,0,No,No,Yes,No,Fiber optic,No,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1,1 - 12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,Yes,Yes,DSL,Yes,No,...,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,0,13 - 24
7039,2234-XADUH,Female,0,Yes,Yes,Yes,Yes,Fiber optic,No,Yes,...,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,0,61 - 72
7040,4801-JZAZL,Female,0,Yes,Yes,No,No phone service,DSL,Yes,No,...,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,0,1 - 12
7041,8361-LTMKD,Male,1,Yes,No,Yes,Yes,Fiber optic,No,No,...,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,1,1 - 12


In [149]:
telco_data= df.copy()

In [150]:
telco_data = telco_data.drop('customerID',axis=1)

In [151]:
telco_data = pd.get_dummies(telco_data)
telco_data.head()

,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,0,1,0,0,1,1,0,...,0,0,1,0,1,0,0,0,0,0
1,0,56.95,1889.50,0,0,1,1,0,1,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53.85,108.15,1,0,1,1,0,1,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42.30,1840.75,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,1,1,0,1,0,1,0,...,0,0,1,0,1,0,0,0,0,0


In [15]:
telco_data.dropna(how='any',axis=0,inplace = True)

In [16]:
telco_data.isnull().sum()

SeniorCitizen                              0
MonthlyCharges                             0
TotalCharges                               0
Churn                                      0
gender_Female                              0
gender_Male                                0
Partner_No                                 0
Partner_Yes                                0
Dependents_No                              0
Dependents_Yes                             0
PhoneService_No                            0
PhoneService_Yes                           0
MultipleLines_No                           0
MultipleLines_No phone service             0
MultipleLines_Yes                          0
InternetService_DSL                        0
InternetService_Fiber optic                0
InternetService_No                         0
OnlineSecurity_No                          0
OnlineSecurity_No internet service         0
OnlineSecurity_Yes                         0
OnlineBackup_No                            0
OnlineBack

In [17]:
#Creating X & Y variable
X= telco_data.drop('Churn',axis = 1)
X

,SeniorCitizen,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,1,0,0,1,1,0,1,...,0,0,1,0,1,0,0,0,0,0
1,0,56.95,1889.50,0,1,1,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53.85,108.15,0,1,1,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42.30,1840.75,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,1,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,84.80,1990.50,0,1,0,1,0,1,0,...,0,0,0,1,0,1,0,0,0,0
7039,0,103.20,7362.90,1,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,1
7040,0,29.60,346.45,1,0,0,1,0,1,1,...,0,0,1,0,1,0,0,0,0,0
7041,1,74.40,306.60,0,1,0,1,1,0,0,...,0,0,0,1,1,0,0,0,0,0


In [18]:
y = telco_data['Churn']
y


0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7032, dtype: int32

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

# Logistic Regression

In [20]:
log = LogisticRegression(random_state=100,max_iter=10)


In [21]:
log.fit(X_train,y_train)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(max_iter=10, random_state=100)

In [22]:
y_pred = log.predict(X_test)

In [23]:
y_pred

array([0, 0, 1, ..., 0, 1, 0])

In [24]:
print(classification_report(y_test,y_pred,labels = [0,1]))

              precision    recall  f1-score   support

           0       0.83      0.79      0.81      1044
           1       0.47      0.53      0.50       363

    accuracy                           0.72      1407
   macro avg       0.65      0.66      0.65      1407
weighted avg       0.74      0.72      0.73      1407



In [25]:
print(confusion_matrix(y_test,y_pred))

[[825 219]
 [171 192]]


In [26]:
lr_score= accuracy_score(y_test,y_pred)

In [27]:
sm = SMOTEENN()

In [28]:
X_resampled3, y_resampled3 = sm.fit_resample(X,y)

In [29]:
xr_train3,xr_test3,yr_train3,yr_test3=train_test_split(X_resampled3, y_resampled3,test_size=0.2)

In [30]:
log_smote = LogisticRegression(random_state=100,max_iter=10)


In [31]:
log_smote.fit(xr_train3,yr_train3)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(max_iter=10, random_state=100)

In [32]:
y_pred_smote = log.predict(xr_test3)

In [33]:
print(classification_report(yr_test3,y_pred_smote,labels = [0,1]))

              precision    recall  f1-score   support

           0       0.66      0.90      0.76       495
           1       0.90      0.67      0.77       682

    accuracy                           0.77      1177
   macro avg       0.78      0.79      0.77      1177
weighted avg       0.80      0.77      0.77      1177



In [34]:
print(confusion_matrix(yr_test3,y_pred_smote))

[[447  48]
 [227 455]]


In [35]:
lr_score_smote= accuracy_score(yr_test3,y_pred_smote)

In [36]:
print(lr_score_smote)

0.7663551401869159


# Decision Tree Classifier

In [37]:
model_dt=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth= 5, min_samples_leaf=8)

In [38]:
model_dt.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=5, min_samples_leaf=8, random_state=100)

In [39]:
dt_pred= model_dt.predict(X_test)

In [40]:
dt_pred

array([0, 0, 0, ..., 0, 1, 0])

In [41]:
model_dt.score(X_test,y_test)

0.7882018479033405

In [42]:
accuracy_score(y_test, dt_pred)

0.7882018479033405

In [43]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.83      0.79      0.81      1044
           1       0.47      0.53      0.50       363

    accuracy                           0.72      1407
   macro avg       0.65      0.66      0.65      1407
weighted avg       0.74      0.72      0.73      1407



In [44]:
sm = SMOTEENN()

In [45]:
X_resampled, y_resampled = sm.fit_resample(X,y)

In [46]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [47]:
model_dt_smote=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [48]:
model_dt_smote.fit(xr_train,yr_train)


DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [49]:
yr_predict = model_dt_smote.predict(xr_test)
model_score_r = model_dt_smote.score(xr_test, yr_test)
print(model_score_r)
print(metrics.classification_report(yr_test, yr_predict))

0.9043552519214346
              precision    recall  f1-score   support

           0       0.89      0.89      0.89       514
           1       0.91      0.91      0.91       657

    accuracy                           0.90      1171
   macro avg       0.90      0.90      0.90      1171
weighted avg       0.90      0.90      0.90      1171



In [50]:
print(metrics.confusion_matrix(yr_test, yr_predict))

[[458  56]
 [ 56 601]]


# Random Forest Classifier

In [51]:
from sklearn.ensemble import RandomForestClassifier

In [52]:
model_rf=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [53]:
model_rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [54]:
rf_pred=model_rf.predict(X_test)

In [55]:
model_rf.score(X_test,y_test)

0.8052594171997157

In [56]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.83      0.79      0.81      1044
           1       0.47      0.53      0.50       363

    accuracy                           0.72      1407
   macro avg       0.65      0.66      0.65      1407
weighted avg       0.74      0.72      0.73      1407



In [57]:
X_resampled1, y_resampled1 = sm.fit_resample(X,y)

In [58]:
xr_train1,xr_test1,yr_train1,yr_test1=train_test_split(X_resampled1, y_resampled1,test_size=0.2)

In [59]:
model_rf_smote=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=10, min_samples_leaf=8)

In [87]:
model_rf_smote.fit(xr_train1,yr_train1)

RandomForestClassifier(max_depth=10, min_samples_leaf=8, random_state=100)

In [111]:
yr_predict1 = model_rf_smote.predict(X)

In [89]:
rf_prob = model_rf_smote.predict_proba(X)[:,1]

In [117]:
#telco_data['Predictions'] = yr_predict1


In [118]:
#telco_data['Probabilities'] = rf_prob

In [121]:
#telco_data = telco_data.drop(['Predictions','Probabilities'],axis = 1)


In [122]:
telco_data.head()

,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,0,1,0,0,1,1,0,...,0,0,1,0,1,0,0,0,0,0
1,0,56.95,1889.50,0,0,1,1,0,1,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53.85,108.15,1,0,1,1,0,1,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42.30,1840.75,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,1,1,0,1,0,1,0,...,0,0,1,0,1,0,0,0,0,0


In [91]:
model_score_r1 = model_rf_smote.score(xr_test1, yr_test1)

In [115]:
print(model_score_r1)
#print(metrics.classification_report(xr_test1, yr_test1))

0.9510309278350515


In [116]:
#print(metrics.confusion_matrix(yr_test1, yr_predict1))

# XGBoost Classifier

In [70]:
from xgboost import XGBClassifier

In [71]:
xg = XGBClassifier()

In [72]:
xg.fit(X_train,y_train)
xg_pred = xg.predict(X_test)

C:\Users\ASUS\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:30:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [73]:
xg.score(X_test,y_test)

0.7846481876332623

In [74]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.83      0.79      0.81      1044
           1       0.47      0.53      0.50       363

    accuracy                           0.72      1407
   macro avg       0.65      0.66      0.65      1407
weighted avg       0.74      0.72      0.73      1407



In [75]:
X_resampled2, y_resampled2 = sm.fit_resample(X,y)

In [76]:
xr_train2,xr_test2,yr_train2,yr_test2=train_test_split(X_resampled2, y_resampled2,test_size=0.2)

In [77]:
xg_smote = XGBClassifier()

In [78]:
xg_smote.fit(xr_train2,yr_train2)

C:\Users\ASUS\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:31:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [124]:
#xg_predict1 = xg_smote.predict(xr_test2)
xg_predict1 = xg_smote.predict(X)

In [82]:
xg_smote_score = xg_smote.score(xr_test2, yr_test2)
print(xg_smote_score)
print(metrics.classification_report(yr_test2, xg_predict1))

0.9648370497427101
              precision    recall  f1-score   support

           0       0.96      0.95      0.96       505
           1       0.97      0.97      0.97       661

    accuracy                           0.96      1166
   macro avg       0.96      0.96      0.96      1166
weighted avg       0.96      0.96      0.96      1166



In [98]:
xg_prob = xg_smote.predict_proba(X)[:,1]

In [152]:
df['Predictions'] = xg_predict1
df['Probabilities'] = xg_prob

In [153]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_group,Predictions,Probabilities
0,7590-VHVEG,Female,0,Yes,No,No,No phone service,DSL,No,Yes,...,No,Month-to-month,Yes,Electronic check,29.85,29.85,0,1 - 12,1,0.996383
1,5575-GNVDE,Male,0,No,No,Yes,No,DSL,Yes,No,...,No,One year,No,Mailed check,56.95,1889.50,0,25 - 36,0,0.000120
2,3668-QPYBK,Male,0,No,No,Yes,No,DSL,Yes,Yes,...,No,Month-to-month,Yes,Mailed check,53.85,108.15,1,1 - 12,1,0.977313
3,7795-CFOCW,Male,0,No,No,No,No phone service,DSL,Yes,No,...,No,One year,No,Bank transfer (automatic),42.30,1840.75,0,37 - 48,0,0.003394
4,9237-HQITU,Female,0,No,No,Yes,No,Fiber optic,No,No,...,No,Month-to-month,Yes,Electronic check,70.70,151.65,1,1 - 12,1,0.999712


In [125]:
telco_data['Predictions'] = xg_predict1
telco_data['Probabilities'] = xg_prob

In [127]:
telco_data.head()

,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72,Predictions,Probabilities
0,0,29.85,29.85,0,1,0,0,1,1,0,...,1,0,1,0,0,0,0,0,1,0.996383
1,0,56.95,1889.50,0,0,1,1,0,1,0,...,0,1,0,0,1,0,0,0,0,0.000120
2,0,53.85,108.15,1,0,1,1,0,1,0,...,0,1,1,0,0,0,0,0,1,0.977313
3,0,42.30,1840.75,0,0,1,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0.003394
4,0,70.70,151.65,1,1,0,1,0,1,0,...,1,0,1,0,0,0,0,0,1,0.999712


In [83]:
#model_eval = pd.DataFrame(index = ["Model"] ,columns = ['Score'])

In [94]:
data = {'Model': ["Logistic Regression","Decision Tree","Random Forest","XG-Boost"],
         'Score':[lr_score,model_score_r,model_score_r1,
                 xg_smote_score]}

In [95]:
model_eval = pd.DataFrame(data)

In [96]:
model_eval

,Model,Score
0,Logistic Regression,0.722814
1,Decision Tree,0.904355
2,Random Forest,0.951031
3,XG-Boost,0.964837


In [126]:
#model_eval.to_csv('model.csv')

In [128]:
xg_smote.feature_importances_

array([0.00171006, 0.00331334, 0.00451868, 0.0025207 , 0.0032064 ,
       0.00437494, 0.00315219, 0.00219463, 0.00416798, 0.00250605,
       0.00178287, 0.00269633, 0.        , 0.00184694, 0.08527683,
       0.02671045, 0.09289747, 0.00467819, 0.        , 0.00784507,
       0.00152969, 0.        , 0.00416574, 0.00252717, 0.        ,
       0.00348202, 0.00497422, 0.        , 0.00322585, 0.00491606,
       0.        , 0.00168964, 0.00402782, 0.        , 0.00498949,
       0.4187327 , 0.05977501, 0.03280149, 0.01511174, 0.00994731,
       0.00587705, 0.00712242, 0.00342404, 0.00385619, 0.11629515,
       0.00305949, 0.00269874, 0.0026973 , 0.00671195, 0.02096261],
      dtype=float32)

In [129]:
weights = pd.DataFrame(xg_smote.feature_importances_, index = X.columns.values)

In [130]:
weights

,0
SeniorCitizen,0.001710
MonthlyCharges,0.003313
TotalCharges,0.004519
gender_Female,0.002521
gender_Male,0.003206
Partner_No,0.004375
Partner_Yes,0.003152
Dependents_No,0.002195
Dependents_Yes,0.004168
PhoneService_No,0.002506


In [131]:
#weights.to_csv('weights.csv')

In [132]:
#telco_data.to_csv('telco_data.csv')

In [154]:
#df.to_csv('telecom_data.csv')